<a href="https://colab.research.google.com/github/Jinukki/KJU/blob/master/Reinforce_ch3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch3. 정책 그래디언트
---
#### 모든 강화학습의 <b>최종 목표</b> $\rightarrow$ 누적 보상을 최대화하는 <b>최적 정책</b>을 구하는 것

<br>

#### <b>정책 그래디언트</b> 
1. 정책을 파라미터화
2. 누적 보상과 정책 파라미터 간의 함수 관계 구축
3. 최적화 방법을 사용해 정책 파라미터 계산

<br>

#### 즉, 반환값의 기댓값으로 나타나는 목적함수 J를 최대로 만드는 정책 파라미터 $\theta$를 구하는 것
##### (여기서 정책은 $\pi_\theta(u_t \mid x_t)$)
$$\theta^*=argmaxJ(\theta)$$
$$J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T\gamma^tr(x_t,u_t)]$$

<br>

##### ( <i>$G_0=\sum_{t=0}^T\gamma^tr(x_t,u_t)$는 전체 보상의 총합을 의미하고, 추가적으로 $G_t=\sum_{k=t}^T\gamma^{k-t}r(x_t,u_t)$는 k=t부터의 예정 보상의 총합을 의미 </i> )<br>

<br>

#### 정책은 보통 신경망으로 파라미터화하여 정책 신경망(policy neural network)이라 부르고,<br> 파라미터 $\theta$는 신경망의 모든 가중치이다.

![KakaoTalk_20200527_012233669_02](https://user-images.githubusercontent.com/52277776/82925426-a89ece00-9fb8-11ea-9d53-750175255bfa.jpg)

<br>

#### 앞에서의 MDP 프로세스를 다시 나타내면 아래 그림과 같다.

![KakaoTalk_20200527_012233669_01](https://user-images.githubusercontent.com/52277776/82925444-adfc1880-9fb8-11ea-943a-ab4e58f965d0.jpg)

<br>

#### 베이즈정리를 이용하여 $p_\theta(\tau)$를 정리하면 아래와 같은데,
$$p_\theta(\tau)=p_\theta(x_0, u_0, x_1, u_1, \dots, x_T, u_T)$$
$$=p(x_0)p_\theta(u_0, x_1, \dots, x_T, u_T \mid x_0)=\dots$$
$$=p(x_0)p_\theta(u_0 \mid x_0)p(x_1 \mid x_0, u_0)p_\theta(u_1 \mid x_0, u_0, x_1)p(x_2 \mid x_0, u_0, x_1, u_1)p_\theta(u_2, \dots, x_T, u_T \mid x_0, u_0, x_1, u_1, x_2)$$

<br>

#### 여기서 아래 첨자가 없는 확률밀도함수들은 정책 $\pi_\theta$와 무관한 환경의 모델이다.<br>추가적으로 <b>마르코프 시퀀스 가정(미래의 상태와 행동은 현재의 상태와 행동에만 영향받음)</b>에 의해,
$$p_\theta(u_1 \mid x_0, u_0, x_1)=\pi_\theta(u_1 \mid x_1)$$
$$p(x_2 \mid x_0, u_0, x_1, u_1)=p(x_2 \mid x_1, u_1)$$

<br>

#### 등의 변형이 가능하므로 최종적으로 정리한 식은 다음과 같다.
$$p_\theta(\tau)=p(x_0)\prod_{t=0}^T\pi_\theta(u_t \mid x_t)p(x_{t+1} \mid x_t, u_t)$$

<br>

#### 한편, 목적함수는 궤도 분리 및 베이즈 정리를 이용한 전개하여 상태가치 함수와 관계가 있음을 알 수 있다.
$$J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T\gamma^tr(x_t,u_t)]$$
$$=\int_{\tau}p_\theta(\tau)(\sum_{t=0}^T\gamma^tr(x_t,u_t))d\tau$$
$$=\int_{x_0}[\int_{\tau_{u_0:u_T}}p_\theta(\tau_{u_0:u_T} \mid x_0)(\sum_{t=0}^T\gamma^tr(x_t,u_t))d\tau_{u_0:u_T}]p(x_0)dx_0$$
$$=\int_{x_0}V^\pi_\theta(x_0)p(x_0)dx_0=E_{x_0 \sim p(x_0)}[V^\pi_\theta(x_0)]$$

<br>

#### <i>즉, 목적함수는 초기 상태변수 $x_0$에 대한 <b>상태가치의 평균값이 된다.</i>

<br>

---


## 정책 그래디언트
#### 목적함수를 최대로 만드는 파라미터 $\theta$의 계산을 위해 목적함수를 $\theta$로 미분하면 다음과 같다.
$$\frac{\partial J(\theta)}{\partial\theta}=\nabla_\theta J(\theta)=\int_{\tau}\nabla_\theta p_\theta(\tau)\sum_{t=0}^T\gamma^tr(x_t, u_t)d\tau$$

<br>

#### 여기서 $\nabla_\theta logp_\theta(\tau)=\frac{\nabla_\theta p_\theta(\tau)}{p_\theta(\tau)}$ 식을 적용하면
$$=\int_{\tau}p_\theta(\tau)\nabla_\theta logp_\theta(\tau)\sum_{t=0}^T\gamma^tr(x_t, u_t)d\tau$$

<br>

#### 이고, 위에서 구한 $p_\theta(\tau)$에 대한 식을 이용하여 $\nabla_\theta logp_\theta(\tau)$를 전개하면 다음과 같다.
$$\nabla_\theta logp_\theta(\tau)=\nabla_\theta log(p(x_0)\prod_{t=0}^T\pi_\theta(u_t \mid x_t)p(x_{t+1} \mid x_t, u_t))$$
$$=\nabla_\theta(logp(x_0)+\sum_{t=0}^Tlog\pi_\theta(u_t \mid x_t)+logp(x_{t+1} \mid x_t, u_t))$$
$$=\sum_{t=0}^T\nabla_\theta log\pi_\theta(u_t \mid x_t)$$

<br>

#### 이를 위 식에 대입하면 다음과 같이 정리된다.
$$\nabla_\theta J(\theta)=E_{\tau \sim p_\theta(\tau)}[(\sum_{t=0}^T \nabla_\theta log\pi_\theta(u_t \mid x_t))(\sum_{t=0}^T\gamma^tr(x_t, u_t))]$$

<br>

#### 위 식에서 환경의 모델을 나타내는 상태천이 확률밀도함수 항이 존재하지 않는 것을 볼 수 있고, 즉 <b>목적함수의 그래디언트</b>를 이용하는 방법은 <b>모델 프리 방법</b>이라는 것을 알 수 있다.

<br>

#### 위 식에 인과성을 고려하여 수정하면 아래와 같고,
$$\nabla_\theta J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T( \nabla_\theta log\pi_\theta(u_t \mid x_t)(\sum_{k=t}^T\gamma^kr(x_k, u_k)))]$$
$$=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T (\gamma^t\nabla_\theta log\pi_\theta(u_t \mid x_t)(\sum_{k=t}^T\gamma^{k-t}r(x_k, u_k)))]$$

<br>

#### 감가율의 특성을 고려하여(너무 작거나 클 때의 영향) 실용적인 형태로 적용하면 다음과 같다.
$$\nabla_\theta J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T( \nabla_\theta log\pi_\theta(u_t \mid x_t)(\sum_{k=t}^T\gamma^{k-t}r(x_k, u_k)))]$$

<br>

#### 이 식을 이용하여 목적함수를 최대로 하는 파라미터 $\theta$는 경사상승법으로 구할 수 있다.
$$\theta \leftarrow \theta + \alpha\nabla_\theta J(\theta)$$

<br>

---



## Reinforce 알고리즘
#### 정책 그래디언트 계산에서, 기댓값의 직접적 계산은 어려우므로 샘플링 후 평균값을 구하여 근사적으로 계산한다.
$$E_{\tau \sim p_\theta(\tau)}[\cdot]\sim \frac{1}{M}\sum_{m=1}^M[\cdot]$$

<br>

#### 목적함수의 그래디언트는 다음과 같이 근사적으로 추정 가능하다.
$$\nabla_\theta J(\theta)\sim \frac{1}{M}\sum_{m=1}^M[\sum_{t=0}^T( \nabla_\theta log\pi_\theta(u_t^{(m)} \mid x_t^{(m)})(\sum_{k=t}^T\gamma^{k-t}r(x_k^{(m)}, u_k^{(m)})))]$$
$$=\nabla_\theta \frac{1}{M}\sum_{m=1}^M[\sum_{t=0}^T( \nabla_\theta log\pi_\theta(u_t^{(m)} \mid x_t^{(m)})G_t^{(m)})]$$

<br>

##### <i>($G_t^{(m)}$은 $\theta$의 함수가 아님 )</i>

<br>

#### 또한 정책 $\pi_\theta$로 구성된 신경망에서 손실함수는
$$loss=-\sum_{t=0}^T(log\pi_\theta(u_t^{(m)} \mid x_t^{(m)})G_t^{(m()})$$

<br>

#### 즉, 파라미터 $\theta$는 다음과 같이 구한다.
$$\theta \leftarrow \theta - \alpha\nabla_\theta log\pi_\theta(u_t^{(m)} \mid x_t^{(m)})G_t^{(m()})$$

<br>

#### 지금까지의 Reinforce 프로세스를 정리하면 다음 그림과 같다.

![KakaoTalk_20200527_012233669](https://user-images.githubusercontent.com/52277776/82925453-b18f9f80-9fb8-11ea-98f2-4d9cfe467795.jpg)

<br>

---


### Reinforce 알고리즘의 문제점
1. 에피소드가 끝나야 정책 업데이트 가능(몬테카를로 정책)
$\rightarrow$ 오랜 학습 시간 요구
2. 매우 큰 그래디언트의 분산(반환값의 변화량이 상당히 큼)
3. 정책 업데이트를 위해서는 해당 정책 실행으로 발생하는 샘플 필요(온-폴리시 방법)

### $\rightarrow$ 현재 Reinforce 알고리즘은 사용되지 않지만, 다른 정책 그래디언트 기반 알고리즘의 기초가 된다는 점에서 의의